## Astronet

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
import tensorflow as tf

#### Conv. local view

In [ ]:
input_lv=Input(shape=(201,1))

#convolution layer
conv1_lv=tf.keras.layers.Conv1D(filters = 16, kernel_size = 5, padding = "same", activation = "relu")(input_lv)
conv2_lv=tf.keras.layers.Conv1D(filters = 16, kernel_size = 5, padding = "same", activation = "relu")(conv1_lv)

#maxpool layer
MaxPool1_lv=tf.keras.layers.MaxPooling1D(pool_size = 7, strides = 2)(conv2_lv)

#convolution layer
conv3_lv=tf.keras.layers.Conv1D(filters = 32, kernel_size = 5, padding = "same", activation = "relu")(MaxPool1_lv)
conv4_lv=tf.keras.layers.Conv1D(filters = 32, kernel_size = 5, padding = "same", activation = "relu")(conv3_lv)

#maxpool layer
MaxPool2_lv=tf.keras.layers.MaxPooling1D(pool_size = 7, strides = 2)(conv4_lv)

Flatten_lv=tf.keras.layers.Flatten()(MaxPool2_lv)

#### Conv. global view

In [ ]:
input_gv=Input(shape=(2001,1))

#convolution layer
conv1_gv=tf.keras.layers.Conv1D(filters = 16, kernel_size = 5, padding = "same", activation = "relu")(input_gv)
conv2_gv=tf.keras.layers.Conv1D(filters = 16, kernel_size = 5, padding = "same", activation = "relu")(conv1_gv)

#maxpool layer
MaxPool1_gv=tf.keras.layers.MaxPooling1D(pool_size = 5, strides = 2)(conv2_gv)

#convolution layer
conv3_gv=tf.keras.layers.Conv1D(filters = 32, kernel_size = 5, padding = "same", activation = "relu")(MaxPool1_gv)
conv4_gv=tf.keras.layers.Conv1D(filters = 32, kernel_size = 5, padding = "same", activation = "relu")(conv3_gv)

#maxpool layer
MaxPool2_gv=tf.keras.layers.MaxPooling1D(pool_size = 5, strides = 2)(conv4_gv)

#convolution layer
conv5_gv=tf.keras.layers.Conv1D(filters = 64, kernel_size = 5, padding = "same", activation = "relu")(MaxPool2_gv)
conv6_gv=tf.keras.layers.Conv1D(filters = 64, kernel_size = 5, padding = "same", activation = "relu")(conv5_gv)

#maxpool layer
MaxPool3_gv=tf.keras.layers.MaxPooling1D(pool_size = 5, strides = 2)(conv6_gv)

#convolution layer
conv7_gv=tf.keras.layers.Conv1D(filters = 128, kernel_size = 5, padding = "same", activation = "relu")(MaxPool3_gv)
conv8_gv=tf.keras.layers.Conv1D(filters = 128, kernel_size = 5, padding = "same", activation = "relu")(conv7_gv)

#maxpool layer
MaxPool4_gv=tf.keras.layers.MaxPooling1D(pool_size = 5, strides = 2)(conv8_gv)

#convolution layer
conv9_gv=tf.keras.layers.Conv1D(filters = 256, kernel_size = 5, padding = "same", activation = "relu")(MaxPool4_gv)
conv10_gv=tf.keras.layers.Conv1D(filters = 256, kernel_size = 5, padding = "same", activation = "relu")(conv9_gv)

#maxpool layer
MaxPool5_gv=tf.keras.layers.MaxPooling1D(pool_size = 5, strides = 2)(conv10_gv)

Flatten_gv=tf.keras.layers.Flatten()(MaxPool5_gv)

In [ ]:
#concatenate local view and global view
lv_and_gv=tf.keras.layers.concatenate([Flatten_lv, Flatten_gv],axis=-1)

#### Fully connected layers

In [ ]:
Dense1=tf.keras.layers.Dense(512, activation="relu")(lv_and_gv)
Dense2=tf.keras.layers.Dense(512, activation="relu")(Dense1)
Dense3=tf.keras.layers.Dense(512, activation="relu")(Dense2)
Dense4=tf.keras.layers.Dense(512, activation="relu")(Dense3)
prediction=tf.keras.layers.Dense(1, activation="sigmoid")(Dense4)

In [ ]:
model=Model(inputs=[input_lv,input_gv],outputs=prediction)

#### COMPILATION AND TRANING

In [ ]:
#path where the data of accuracy, loss, etc. in each step will be saved
name="S&B-{}".format(np.int(time.time()))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs\{}".format(name), histogram_freq=1)



#Adam optimizer with the different parameters
adam=tf.keras.optimizers.Adam(learning_rate=1e-05,beta_1=0.9,beta_2=0.999,epsilon=1e-08)

#COMPILATION
model.compile(optimizer=adam,
              loss="binary_crossentropy",
              metrics=["accuracy"])

#training
history = model.fit([lv_train_curve, gv_train_curve], 
                    train_label2, 
                    batch_size=64, 
                    epochs=50, 
                    validation_data=([lv_val_curve,gv_val_curve],val_label2),
                    callbacks=[tensorboard_callback]
                   )

## Final configurations

### GLOBAL & LOCAL VIEW + AUX. PARAMETERS (DEPTH & SNR)

In [ ]:
def HP_shallue_param(convs_gv,convs_lv,denses,units):
    '''
    Create the structure of a ANN with two different inputs (based of Astronet)
    
    convs_gv= # of conv. layers in the global view
    convs_lv= # of conv. layers in the local view
    denses= # of dense layers
    units= # of units in the dense layer
    
    it returns the output layer and the different inputs (only 1 aux. param.)
    '''
    
    input_lv=Input(shape=(201,1))
    Input_prev=input_lv
    
    num_filters=16 #parameter conv. layer

    # local view. We want a conf. such as: 2 conv. + Maxpool + ...(repetiton)
    for n_c_lv in range(convs_lv):
        if n_c_lv%2==0 and n_c_lv!=convs_lv and n_c_lv!=0:
            num_filters=num_filters*2 
            MaxPool_lv=tf.keras.layers.MaxPooling1D(pool_size = 7, strides = 2)(Input_prev)
            conv_lv=tf.keras.layers.Conv1D(filters = num_filters, kernel_size = 5,
                                           padding = "same", activation = "relu")(MaxPool_lv)        
            Input_prev=conv_lv
        else:
            conv_lv=tf.keras.layers.Conv1D(filters = num_filters, kernel_size = 5, 
                                           padding = "same", activation = "relu")(Input_prev)
            Input_prev=conv_lv
    #final MaxPool       
    if convs_lv%2==0:
        MaxPool_lv=tf.keras.layers.MaxPooling1D(pool_size = 7, strides = 2)(Input_prev)       
    Flatten_lv=tf.keras.layers.Flatten()(MaxPool_lv)
    

    input_gv=Input(shape=(2001,1))
    Input_prev=input_gv   
    num_filters=16 
    
    #global view. We want a conf. such as: 2 conv. + Maxpool + ...(repetiton)
    for n_c_gv in range(convs_gv):
        if n_c_gv%2==0 and n_c_gv!=convs_gv and n_c_gv!=0:
            num_filters=num_filters*2 
            MaxPool_gv=tf.keras.layers.MaxPooling1D(pool_size = 5, strides = 2)(Input_prev)

            conv_gv=tf.keras.layers.Conv1D(filters = num_filters, kernel_size = 5, 
                                           padding = "same", activation = "relu")(MaxPool_gv)

            Input_prev=conv_gv

        else:
            conv_gv=tf.keras.layers.Conv1D(filters = num_filters, kernel_size = 5, 
                                           padding = "same", activation = "relu")(Input_prev)
            Input_prev=conv_gv  
    #final MaxPool
    if convs_gv%2==0:
        MaxPool_gv=tf.keras.layers.MaxPooling1D(pool_size = 5, strides = 2)(Input_prev)
    Flatten_gv=tf.keras.layers.Flatten()(MaxPool_gv)
    
    #aux inputs
    aux_input_depth=Input(shape=1)
    aux_input_snr=Input(shape=1)    

    #concatenate all
    lv_and_gv=tf.keras.layers.concatenate([Flatten_lv, Flatten_gv,aux_input_depth,aux_input_snr],axis=-1)
    Input_prev=lv_and_gv
    
    
    #fully connected layers
    for n_dens in range(denses): 
        Dense=tf.keras.layers.Dense(units, activation="relu")(Input_prev)
        Input_prev=Dense
        
    if denses==0:
        prediction=tf.keras.layers.Dense(1, activation="sigmoid")(Input_prev)
    else:
        prediction=tf.keras.layers.Dense(1, activation="sigmoid")(Dense)
    
    return prediction,input_gv,input_lv,aux_input_depth, aux_input_snr

In [ ]:
gvconv=10
lvconv=4
n_dense=4
n_units=128

#call function
pred,gv,lv,depth,snr=HP_shallue_param(gvconv,lvconv,n_dense,n_units)

#def inputs and outputs of the model
model=Model(inputs=[gv,lv,depth,snr],outputs=pred)

#compilation
model.compile(optimizer=adam,
            loss="binary_crossentropy",
            metrics=["accuracy"])

#path where the data of accuracy, loss, etc. in each step will be saved
name="S&V_prova_GV_{}+LV_{}+aux(depth_snr)_ndense-{}_nunit-{}_{}".format(gvconv,lvconv,n_dense,n_units,np.int(time.time()))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="model_GV_LV_P\{}".format(name), histogram_freq=1)

#train
history = model.fit([gv_train_curve,lv_train_curve,train_TCE_depth,train_model_snr], 
                    train_label2, 
                    batch_size=64, 
                    epochs=50, 
                    validation_data=([gv_val_curve,lv_val_curve,val_TCE_depth,val_model_snr],val_label2),
                    callbacks=[tensorboard_callback]
                   )

#evaluation and prediction
loss, acc = model.evaluate([gv_test_curve,lv_test_curve,test_TCE_depth,test_model_snr],  test_label2, verbose=2)
predictions=model.predict([gv_test_curve,lv_test_curve,test_TCE_depth,test_model_snr])

### GLOBAL & LOCAL VIEW + AUX. PARAMETER (DEPTH)

In [ ]:
def HP_shallue_depth(convs_gv,convs_lv,denses,units):
    '''
    Create the structure of a ANN with two different inputs (based of Astronet)
    
    convs_gv= # of conv. layers in the global view
    convs_lv= # of conv. layers in the local view
    denses= # of dense layers
    units= # of units in the dense layer
    
    it returns the output layer and the different inputs (2 aux. param.)
    '''
    
    input_lv=Input(shape=(201,1))
    Input_prev=input_lv
    num_filters=16

    # local view. We want a conf. such as: 2 conv. + Maxpool + ...(repetiton)
    for n_c_lv in range(convs_lv):
        if n_c_lv%2==0 and n_c_lv!=convs_lv and n_c_lv!=0:
           
            num_filters=num_filters*2
            MaxPool_lv=tf.keras.layers.MaxPooling1D(pool_size = 7, strides = 2)(Input_prev)
            conv_lv=tf.keras.layers.Conv1D(filters = num_filters, kernel_size = 5,
                                           padding = "same", activation = "relu")(MaxPool_lv)
            
            
            Input_prev=conv_lv

        else:
            conv_lv=tf.keras.layers.Conv1D(filters = num_filters, kernel_size = 5, 
                                           padding = "same", activation = "relu")(Input_prev)
            Input_prev=conv_lv
            
    #final MaxPool
    if convs_lv%2==0:
        MaxPool_lv=tf.keras.layers.MaxPooling1D(pool_size = 7, strides = 2)(Input_prev)       
    Flatten_lv=tf.keras.layers.Flatten()(MaxPool_lv)
    
    
    input_gv=Input(shape=(2001,1))
    Input_prev=input_gv   
    num_filters=16
    
    #global view. We want a conf. such as: 2 conv. + Maxpool + ...(repetiton)
    for n_c_gv in range(convs_gv):
        if n_c_gv%2==0 and n_c_gv!=convs_gv and n_c_gv!=0:

            num_filters=num_filters*2

            MaxPool_gv=tf.keras.layers.MaxPooling1D(pool_size = 5, strides = 2)(Input_prev)
            conv_gv=tf.keras.layers.Conv1D(filters = num_filters, kernel_size = 5, 
                                           padding = "same", activation = "relu")(MaxPool_gv)

            Input_prev=conv_gv
            

        else:

            conv_gv=tf.keras.layers.Conv1D(filters = num_filters, kernel_size = 5, 
                                           padding = "same", activation = "relu")(Input_prev)
            
            Input_prev=conv_gv
    
    #final MaxPool
    if convs_gv%2==0:
        MaxPool_gv=tf.keras.layers.MaxPooling1D(pool_size = 5, strides = 2)(Input_prev)
    Flatten_gv=tf.keras.layers.Flatten()(MaxPool_gv)
    
    #aux input
    aux_input_depth=Input(shape=1)   

    #concatenate all
    lv_and_gv=tf.keras.layers.concatenate([Flatten_lv, Flatten_gv,aux_input_depth],axis=-1)
    Input_prev=lv_and_gv
    
    #fully connected layers
    for n_dens in range(denses): 
        Dense=tf.keras.layers.Dense(units, activation="relu")(Input_prev)
        Input_prev=Dense
        
    if denses==0:
        prediction=tf.keras.layers.Dense(1, activation="sigmoid")(Input_prev)
    else:
        prediction=tf.keras.layers.Dense(1, activation="sigmoid")(Dense)
    
    return prediction,input_gv,input_lv,aux_input_depth

In [ ]:
gvconv=10
lvconv=4
n_dense=6
n_units=256

#call function
pred,gv,lv,depth=HP_shallue_depth(gvconv,lvconv,n_dense,n_units)

#def inputs and outputs of the model
model=Model(inputs=[gv,lv,depth],outputs=pred)

#compilation
model.compile(optimizer=adam,
            loss="binary_crossentropy",
            metrics=["accuracy"])

#path where the data of accuracy, loss, etc. in each step will be saved
name="S&V_prova_GV_{}+LV_{}+aux(depth)_ndense-{}_nunit-{}_{}".format(gvconv,lvconv,n_dense,n_units,np.int(time.time()))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="model_GV_LV_D\{}".format(name), histogram_freq=1)

#train
history = model.fit([gv_train_curve,lv_train_curve,train_TCE_depth], 
                    train_label2, 
                    batch_size=64, 
                    epochs=50,
                    validation_data=([gv_val_curve,lv_val_curve,val_TCE_depth],val_label2),
                    callbacks=[tensorboard_callback]
                   )

#evaluation and prediction
loss, acc = model.evaluate([gv_test_curve,lv_test_curve,test_TCE_depth],  test_label2, verbose=2)
predictions=model.predict([gv_test_curve,lv_test_curve,test_TCE_depth])
